In [ ]:
## environment preparation
##============================
%reset -f
%connect_info #might be used for "jupyter-console"

In [ ]:
## Import libraries
##============================

import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
## Change to the directory with data
##============================

#Change the directory here:
dir_data = "demo_run/"

path_ipynb = os.getcwd()
path_base = path_ipynb + '/../../data/readback/'
pathW = path_base + dir_data
os.chdir(pathW)
print(os.getcwd())

files_list = os.listdir(pathW)
files_list.sort()

In [ ]:
## Lets see the files with data
##============================

files_list

In [ ]:
## Aggregate data from different 
## files into a single DataFrame
##============================

Xdf = pd.DataFrame([], columns=["time_ms", 
                                "B102NO2", "B302C2H5OH", "B502VOC", "B702CO", 
                                "TdegC", "RH", 
                                "label", "add_label", "condition"])

for fileN in files_list:
    file_info = fileN.split("_")
    label_c = file_info[0]
    add_label_c = file_info[1]
    condition_c = file_info[2]
    
    df_c = pd.read_csv(fileN)
    
    feature_add_dict = {
        "label": label_c,
        "add_label": add_label_c,
        "condition": condition_c,
    } 
    df_c_upd = pd.concat([df_c, pd.DataFrame(feature_add_dict, index=df_c.index)], axis=1)
    Xdf = pd.concat([Xdf, df_c_upd], axis=0)


In [ ]:
## Lets see the DataFrame
##============================

Xdf.head()

In [ ]:
## The labels we have
##============================

Xdf.label.unique().tolist()

In [ ]:
## Aggregate labels to make plots
##============================

Xdf["label_label"] = Xdf[["label","add_label"]]\
                        .agg("_".join, axis=1)\
                        .replace("[0-9]", "", regex=True)

In [ ]:
## See a sample of it
##============================

Xdf.sample()

In [ ]:
## Plot the data
##============================

labels = Xdf.label_label.unique().tolist()
conditions = Xdf.condition.unique().tolist()
figure_scale = 1
plt.figure(figsize=(figure_scale*15, figure_scale*len(labels)*4))
plot_c = 0

for label_c in labels:
    for condition_c in conditions:
        Xplot = Xdf.loc[
            (Xdf.label_label==label_c) & (Xdf.condition==condition_c),
            :]
        plot_c += 1
        plt.subplot(len(labels), len(conditions), plot_c)
        plt.plot(Xplot.time_ms, Xplot.iloc[:,1:7], '.')
        plt.legend(Xplot.iloc[:,1:7].columns.to_list())
        plt.title(" ".join([label_c, condition_c]), color="red")
        plt.tick_params(axis='x', colors='red')
        plt.tick_params(axis='y', colors='red')
        plt.grid()